In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import metrics

In [2]:
# Read the clinical data
df1 = pd.read_excel('1_Clinical data AH_AUD_HC Bernd_bei_updated08012019.xlsx')
df = df1[['subject_id',
          'age', 'alt', 'ast', 'creatinine_value_mg_dl',
          'bilirubin_value_mg_dl', 'platelets_value_10to9_l', 'wbc_10to9_l', 'alk_phos',
          'albumin_value_g_dl', 'sodium', 'inr', 'dialysis_required',
          'ThirtyMo', 'NinetyMo']]

df_30 = df[pd.notnull(df['ThirtyMo'])].drop(['NinetyMo'], axis = 1)
df_90 = df[pd.notnull(df['NinetyMo'])].drop(['ThirtyMo'], axis = 1)

# Fill dialysis
df_30.fillna({'dialysis_required': 0}, inplace = True)
df_90.fillna({'dialysis_required': 0}, inplace = True)

# Calculate MELD score (2016) before imputation
df_30['MELD'] = df_30.apply(lambda row:
                            9.57 * np.log(np.where(row.dialysis_required == 0,
                                                   np.minimum(np.maximum(row.creatinine_value_mg_dl, 1.0), 4.0),
                                                   4)) +
                            3.78 * np.log(np.maximum(row.bilirubin_value_mg_dl, 1.0)) +
                            11.2 * np.log(np.maximum(row.inr, 1.0)) +
                            6.43,
                            axis = 1)
df_30['MELD_2016'] = df_30.apply(lambda row:
                                 np.where(row.MELD > 11,
                                          row.MELD +
                                          1.32 * (137 - np.minimum(np.maximum(row.sodium, 125), 137)) -
                                          0.033 * row.MELD * (137 - np.minimum(np.maximum(row.sodium, 125), 137)),
                                          row.MELD),
                                 axis = 1)

df_90['MELD'] = df_90.apply(lambda row:
                            9.57 * np.log(np.where(row.dialysis_required == 0,
                                                   np.minimum(np.maximum(row.creatinine_value_mg_dl, 1.0), 4.0),
                                                   4)) +
                            3.78 * np.log(np.maximum(row.bilirubin_value_mg_dl, 1.0)) +
                            11.2 * np.log(np.maximum(row.inr, 1.0)) +
                            6.43,
                            axis = 1)
df_90['MELD_2016'] = df_90.apply(lambda row:
                                 np.where(row.MELD > 11,
                                          row.MELD +
                                          1.32 * (137 - np.minimum(np.maximum(row.sodium, 125), 137)) -
                                          0.033 * row.MELD * (137 - np.minimum(np.maximum(row.sodium, 125), 137)),
                                          row.MELD),
                                 axis = 1)

df_30 = df_30.drop(['dialysis_required'], axis = 1)
df_90 = df_90.drop(['dialysis_required'], axis = 1)

In [3]:
# Read fungi data and merge it with clinical data
df2 = pd.read_excel('2_Fungi - updated on 03312020.xlsx')
na_removing_list2 = list(df2.columns)
na_removing_list2.pop(0)

df_30 = df_30.merge(df2, how = 'left', on = 'subject_id')
df_90 = df_90.merge(df2, how = 'left', on = 'subject_id')

In [4]:
# Observations missing rate
print(np.round(1 - df_30.dropna().shape[0] / df_30.shape[0], decimals = 2))
print(np.round(1 - df_90.dropna().shape[0] / df_90.shape[0], decimals = 2))

0.78
0.8


In [5]:
# Drop observations missing with all fungi data
df_30 = df_30.dropna(how = 'all', subset = na_removing_list2)
df_90 = df_90.dropna(how = 'all', subset = na_removing_list2)

# Export the data with MELD score (2016)
df_30.to_csv('MELD2_30.csv', index = False)
df_90.to_csv('MELD2_90.csv', index = False)

In [6]:
print(df_30.shape)
print(df_90.shape)

(54, 96)
(39, 96)


In [7]:
print(df_30.dropna().shape)
print(df_90.dropna().shape)

(46, 96)
(32, 96)


In [8]:
# Observations missing rate
print(np.round(1 - df_30.dropna().shape[0] / df_30.shape[0], decimals = 2))
print(np.round(1 - df_90.dropna().shape[0] / df_90.shape[0], decimals = 2))

0.15
0.18


In [9]:
# Creating feature matrix
X_30 = df_30.drop(['subject_id', 'ThirtyMo'] , axis = 1)
X_90 = df_90.drop(['subject_id', 'NinetyMo'] , axis = 1)

# Creating labels
y_30 = df_30['ThirtyMo']
y_90 = df_90['NinetyMo']

# Saving feature names for later use
X_list_30_raw = list(X_30.columns)
X_list_90_raw = list(X_90.columns)
X_list_30 = [i for i in X_list_30_raw if i not in ('MELD', 'MELD_2016')]
X_list_90 = [i for i in X_list_90_raw if i not in ('MELD', 'MELD_2016')]

In [10]:
# Check whether data is balanced
print(y_30.value_counts())
print(y_90.value_counts())

0    49
1     5
Name: ThirtyMo, dtype: int64
0.0    30
1.0     9
Name: NinetyMo, dtype: int64


In [11]:
# For died within 30 days outcome

# Stratified 5-fold split
skf = StratifiedKFold(n_splits = 5)
skf.get_n_splits(X_30, y_30)

i = 0
for train_index, test_index in skf.split(X_30, y_30):
    X_train_30, y_train_30 = X_30.iloc[train_index], y_30.iloc[train_index]
    X_test_30, y_test_30 = X_30.iloc[test_index], y_30.iloc[test_index]
    
    # Export the raw training data
    train_30 = pd.DataFrame(np.column_stack([X_train_30, y_train_30]))
    train_30.columns = np.concatenate((X_list_30_raw, 'ThirtyMo'), axis = None)
    test_30 = pd.DataFrame(np.column_stack([X_test_30, y_test_30]))
    test_30.columns = np.concatenate((X_list_30_raw, 'ThirtyMo'), axis = None)
    
    train_30.to_csv('train2_30_raw' + str(i) + '.csv', index = False)
    test_30.to_csv('test2_30_raw' + str(i) + '.csv', index = False)
    
    X_train_30 = X_train_30.drop(['MELD', 'MELD_2016'], axis = 1)
    X_test_30 = X_test_30.drop(['MELD', 'MELD_2016'], axis = 1)
    
    # Apply MICE imputation to training data and use the same imputation model to test data
    MICE_imputer = IterativeImputer(sample_posterior = True, min_value = 0, random_state = 0)
    X_train_30_imp = MICE_imputer.fit_transform(X_train_30)
    X_test_30_imp = MICE_imputer.transform(X_test_30)
    
    # Export the training data and test data
    train_30 = pd.DataFrame(np.column_stack([X_train_30_imp, y_train_30]))
    train_30.columns = np.concatenate((X_list_30, 'ThirtyMo'), axis = None)
    test_30 = pd.DataFrame(np.column_stack([X_test_30_imp, y_test_30]))
    test_30.columns = np.concatenate((X_list_30, 'ThirtyMo'), axis = None)
    
    train_30.to_csv('train2_30_imp' + str(i) + '.csv', index = False)
    test_30.to_csv('test2_30_imp' + str(i) + '.csv', index = False)
    
    i += 1

In [12]:
# For died within 90 days outcome

# Stratified 5-fold split
skf = StratifiedKFold(n_splits = 5)
skf.get_n_splits(X_90, y_90)

i = 0
for train_index, test_index in skf.split(X_90, y_90):
    X_train_90, y_train_90 = X_90.iloc[train_index], y_90.iloc[train_index]
    X_test_90, y_test_90 = X_90.iloc[test_index], y_90.iloc[test_index]
    
    # Export the raw training data
    train_90 = pd.DataFrame(np.column_stack([X_train_90, y_train_90]))
    train_90.columns = np.concatenate((X_list_90_raw, 'NinetyMo'), axis = None)
    test_90 = pd.DataFrame(np.column_stack([X_test_90, y_test_90]))
    test_90.columns = np.concatenate((X_list_90_raw, 'NinetyMo'), axis = None)
    
    train_90.to_csv('train2_90_raw' + str(i) + '.csv', index = False)
    test_90.to_csv('test2_90_raw' + str(i) + '.csv', index = False)
    
    X_train_90 = X_train_90.drop(['MELD', 'MELD_2016'], axis = 1)
    X_test_90 = X_test_90.drop(['MELD', 'MELD_2016'], axis = 1)
    
    # Apply MICE imputation to training data and use the same imputation model to test data
    MICE_imputer = IterativeImputer(sample_posterior = True, min_value = 0, random_state = 0)
    X_train_90_imp = MICE_imputer.fit_transform(X_train_90)
    X_test_90_imp = MICE_imputer.transform(X_test_90)
    
    # Export the training data and test data
    train_90 = pd.DataFrame(np.column_stack([X_train_90_imp, y_train_90]))
    train_90.columns = np.concatenate((X_list_90, 'NinetyMo'), axis = None)
    test_90 = pd.DataFrame(np.column_stack([X_test_90_imp, y_test_90]))
    test_90.columns = np.concatenate((X_list_90, 'NinetyMo'), axis = None)
    
    train_90.to_csv('train2_90_imp' + str(i) + '.csv', index = False)
    test_90.to_csv('test2_90_imp' + str(i) + '.csv', index = False)
    
    i += 1

In [13]:
# For died within 30 days outcome

for i in range(5):
    # Import the training data after feature selection
    train_30_sel = pd.read_csv('train2_30_sel' + str(i) + '.csv')
    X_train_30_sel = train_30_sel.drop('ThirtyMo', axis = 1)
    y_train_30 = train_30_sel['ThirtyMo']
    X_sel_list_30 = list(X_train_30_sel.columns)
    
    # Use SMOTE to over sample the minority class for training data
    sm = SMOTE(random_state = 0, k_neighbors = 3)
    X_train_30_res, y_train_30_res = sm.fit_resample(X_train_30_sel, y_train_30)
    
    train_30 = pd.DataFrame(np.column_stack([X_train_30_res, y_train_30_res]))
    train_30.columns = np.concatenate((X_sel_list_30, 'ThirtyMo'), axis = None)
    
    train_30.to_csv('train2_30_' + str(i) + '.csv', index = False)
    
    i += 1

In [14]:
# For died within 90 days outcome

for i in range(5):
    # Import the training data after feature selection
    train_90_sel = pd.read_csv('train2_90_sel' + str(i) + '.csv')
    X_train_90_sel = train_90_sel.drop('NinetyMo', axis = 1)
    y_train_90 = train_90_sel['NinetyMo']
    X_sel_list_90 = list(X_train_90_sel.columns)
    
    # Use SMOTE to over sample the minority class for training data
    sm = SMOTE(random_state = 0, k_neighbors = 3)
    X_train_90_res, y_train_90_res = sm.fit_resample(X_train_90_sel, y_train_90)
    
    train_90 = pd.DataFrame(np.column_stack([X_train_90_res, y_train_90_res]))
    train_90.columns = np.concatenate((X_sel_list_90, 'NinetyMo'), axis = None)
    
    train_90.to_csv('train2_90_' + str(i) + '.csv', index = False)
    
    i += 1